In [2]:
import pandas as pd
import numpy as np

In [4]:
data_df=pd.read_csv("./data/gemstone.csv")
data_df.head(5)

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
X=data_df.drop(labels=["price","id"],axis=1)
X.head(5)

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [6]:
Y=data_df[["price"]]
Y.head(5)

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [7]:
categorical_cols=X.select_dtypes(include='object').columns
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [10]:
numerical_cols=X.select_dtypes(exclude='object').columns
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [11]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [12]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [23]:
num_pipeline=Pipeline(
steps=[
    ('imputer',SimpleImputer()),
    ('scalar',StandardScaler())
]
)

In [24]:
cat_pipeline=Pipeline(
steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('scalar',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
]
)

In [25]:
preprocessor=ColumnTransformer(
[
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
]    
)

In [26]:
## Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [28]:
print(X_train.head(3))
print(X_test.head(3))
print(y_train.head(3))
print(y_test.head(3))


        carat      cut color clarity  depth  table     x     y     z
168192   0.34    Ideal     I    VVS2   60.9   57.0  4.56  4.53  2.76
35202    0.90     Good     E     SI1   63.8   57.0  6.07  6.03  3.87
41091    1.02  Premium     G     VS1   62.7   58.0  6.35  6.39  4.00
        carat        cut color clarity  depth  table     x     y     z
70432    0.53    Premium     E     VS2   60.8   56.0  5.24  5.21  3.19
64839    0.71  Very Good     H     SI1   62.9   57.0  5.67  5.69  3.56
185316   0.30      Ideal     H      IF   62.1   57.0  4.27  4.29  2.66
        price
168192    765
35202    4763
41091    6139
        price
70432    1607
64839    2211
185316    764


In [29]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [30]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,4.0,5.0,5.0
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,1.0,1.0,2.0
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,3.0,3.0,4.0
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,3.0,3.0,3.0
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,4.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,4.0,1.0,2.0
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,4.0,3.0,1.0
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,1.0,3.0,2.0
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,4.0,3.0,4.0


In [31]:
X_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,3.0,1.0,3.0
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,2.0,4.0,2.0
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,4.0,4.0,7.0
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,3.0,3.0,3.0
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,3.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...
58067,1.013460,1.185958,-0.642862,1.003556,1.041924,1.151941,2.0,4.0,3.0
58068,-0.997058,0.260701,-1.164193,-1.141917,-1.126331,-1.108659,4.0,2.0,6.0
58069,-0.197174,-3.347799,1.442462,0.102096,0.071199,-0.224706,3.0,6.0,3.0
58070,-0.824110,-0.201927,-0.121531,-0.853450,-0.881382,-0.876803,4.0,3.0,2.0


In [32]:
"""linear regression
ridge regression
lasso regression
elastic net"""

'linear regression\nridge regression\nlasso regression\nelastic net'

In [33]:
## Model Training
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [34]:
model_1 = LinearRegression()

In [38]:
model_1.fit(X_train,y_train)


LinearRegression()

In [42]:
y_pred1=model_1.predict(X_test)
print("model score",model_1.score)
print("r2 score",r2_score(y_test,y_pred1))
print("MSE",mean_squared_error(y_test,y_pred1))
print("MAE",mean_absolute_error(y_test,y_pred1))
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred1)))


model score <bound method RegressorMixin.score of LinearRegression()>
r2 score 0.9368908248567511
MSE 1028002.7598132559
MAE 674.0255115796833
RMSE 1013.9047094344004


In [43]:
def mod_evo_met(true,predicted):
    R2_SCORE= r2_score(true,predicted)
    MSE= mean_squared_error(true,predicted)
    MAE= mean_absolute_error(true,predicted)
    RMSE= np.sqrt(mean_squared_error(true,predicted))
    return R2_SCORE,MSE,MAE,RMSE


In [61]:
models_list = {'LinearRegression':LinearRegression(), 'Lasso':Lasso(), 'Ridge':Ridge(), 'ElasticNet':ElasticNet()}
print(list(models_list.keys())[0])
print(list(models_list.values())[0])
"""
for i in range(len(models_list)):
    print(models_list[i].keys())
    """

LinearRegression
LinearRegression()


'\nfor i in range(len(models_list)):\n    print(models_list[i].keys())\n    '

In [ ]:
for i in list1:
    model = i
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    R2_SCORE,MSE,MAE,RMSE=mod_evo_met(y_test,y_pred1)